# **Finetuning Vanilla GPT2 using HuggingFace and Tensorflow**

This notebook uses the hugginface finefuning scripts and then uses the TensorFlow version of the genreated models. To generate the texts for a traget character.

First begin setup by cloning transformers repo. We need to store the training script locally as that would be the most easy way.

In [ ]:
#Clone the transformers repo into the notebook
!git clone https://github.com/huggingface/transformers

Change directory location to be in the examples folder and then install any requirements

In [1]:
import os
os.chdir("transformers")
os.chdir("./examples/pytorch/language-modeling")
!Dir

 Volume in drive C is OS
 Volume Serial Number is CAB1-400D

 Directory of C:\Users\purav\Downloads\Study\NLP\Project\Project github\Alpha-Friends\Model\vanilla-gpt2\transformers\examples\pytorch\language-modeling

04/22/2021  12:04 AM    <DIR>          .
04/22/2021  12:04 AM    <DIR>          ..
04/22/2021  12:04 AM           350,616 eval_tmp.txt
04/21/2021  11:47 PM             6,621 README.md
04/21/2021  11:47 PM                54 requirements.txt
04/21/2021  11:47 PM            19,545 run_clm.py
04/21/2021  11:47 PM            18,487 run_clm_no_trainer.py
04/21/2021  11:47 PM            21,035 run_mlm.py
04/21/2021  11:47 PM            20,609 run_mlm_no_trainer.py
04/21/2021  11:47 PM            19,617 run_plm.py
04/22/2021  12:04 AM         3,049,519 train_tmp.txt
               9 File(s)      3,506,103 bytes
               2 Dir(s)  72,743,759,872 bytes free


In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install pyarrow --upgrade

In [ ]:
# Need to install latest transformer packages from github so the scripts will run correctly
! pip install git+git://github.com/huggingface/transformers/

Load the data created previously and split it in to train and test. Change the file name as per requirement.

In [2]:
"""
Now load the data line by line
"""
from sklearn.model_selection import train_test_split

with open('../../../../../../Data/Ross_dialogues_all.txt', 'r',encoding='utf-8') as data:
    dataset = ["<|title|>" + x.strip() for x in data.readlines()]

train, eval = train_test_split(dataset, train_size=.9, random_state=2020)
print("training size:" + str(len(train)))
print("Evaluation size: " + str(len(eval)))

with open('train_tmp.txt', 'w') as file_handle:
    file_handle.write("<|endoftext|>".join(train))

with open('eval_tmp.txt', 'w') as file_handle:
    file_handle.write("<|endoftext|>".join(eval))

training size:2796
Evaluation size: 311


The script below will fine tune GPT2 on your text data that we setup above. This training step will take anywhre from tens of minutes to hours depending on how large your training set is, how many epochs you intend to train on, and if you are using Gpu or not.

In [3]:
!python run_clm.py \
--model_type gpt2 \
--model_name_or_path gpt2 \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--output_dir="../../../..Ross_model" \
--overwrite_output_dir

2021-04-22 00:07:22.725517: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-04-22 00:07:22.726180: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Traceback (most recent call last):
  File "run_clm.py", line 457, in <module>
    main()
  File "run_clm.py", line 182, in main
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
  File "C:\Users\purav\anaconda3\envs\nlp\lib\site-packages\transformers\hf_argparser.py", line 187, in parse_args_into_dataclasses
    obj = dtype(**inputs)
  File "<string>", line 67, in __init__
  File "C:\Users\purav\anaconda3\envs\nlp\lib\site-packages\transformers\training_args.py", line 581, in __post_init__
    "Mixed precision training with AMP or APEX (`--fp16`) and FP16 evaluation can only be used on CUDA devices."
ValueError: Mixed precisi

# **Using the model**
Next lets take the model we just trained and use it to generate some text! We will import the Tensorflow version of the gpt2 language model and set the from_pt flag to True. Then we load a pretrained tokenizer from huggingface. This may take some time to download the tokenizer data.

In [2]:
# setup imports to use the model
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

model = TFGPT2LMHeadModel.from_pretrained("../../../../Ross_model", from_pt=True)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.5.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'lm_head.weight', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.1.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

### To generate only samples from one sentence input use the below cell blocks

We fitst encode the input using the pretrained tokenizer. Next we will use the model to generate the text from our input sample. The parameters I used are based on trial and error from playing around with the huggingface tutorial, https://huggingface.co/blog/how-to-generate, which really goes into great detail on how to go about finding the best parameters for generating text. As well they dive into really good information on what each parameter does and how they play into one another.

In [3]:
# Change the character name here <Ross> to <Character> for the character you trained your model on
input_ids = tokenizer.encode("Oh it's OK. You were worth the wait, and I don't just mean tonight.<Ross>", return_tensors='tf')
generated_text_samples = model.generate(
    input_ids, 
    max_length=150,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)
#Print output for each sequence generated above
for i, beam in enumerate(generated_text_samples):
    print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
    print()

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


0: Oh it's OK. You were worth the wait, and I don't just mean tonight.<Ross>Yeah, of course you were too late for me!<endRoss><Ross>: What about this big fight with her?<endsRach: Come on Rachel, get out there! Get back in your chair!Hey Phoebe!Come here.We're not getting married!!I'll see you at work tomorrow afternoon.Okay, when do we start working on that wedding ring?!When are they supposed to be ready by today anyway??Phyllis got a little busy Monday morning...Oh my God! Is she gonna make dinner while we go?Monica, Monica will have lunch later than any other time before!Wait Chandler

1: Oh it's OK. You were worth the wait, and I don't just mean tonight.<Ross>What?<endMonica>: A little later than we'd have liked, but this is a really cool date.Hey!So you're back from Mexico.Yeah! So what are ya gonna do tomorrow night with my grandmother?!Oh yeah-no. Y'know um...<Richard starts kissing Monica on her pillow>Uh Rach umm, okay if yay, maybe uh...We could actually get that couch here 

### To generate sentences and interact with the model run the following cells

First the system would ask you to start a conversation. After that based on the value of "num_return_sequences" the model system would generate those many reposnses. You will need to select the response you liked the most by entering values from 0-5 as for now but this can be changed. After that you can add your response to the generated text and the model would then take the whole conversation that has happened till now and generate further responses using the previous converation as history in form of input.

In [5]:
flag = True
count =0
# change the start and end str accddingly for different characters
start_str = "<Ross>"
end_str = "<endRoss>"
while flag:
    if count==0:
        X = input("Start the conversation:  ")
        count+=1
    if X =="exit":
        flag=False
        continue

    input_ids = tokenizer.encode(X+" "+start_str, return_tensors='tf')
    generated_text_samples = model.generate(
        input_ids, 
        max_length=150,
        num_return_sequences=5,
        no_repeat_ngram_size=2,
        repetition_penalty=1.5,
        top_p=0.92,
        temperature=.85,
        do_sample=True,
        top_k=125,
        early_stopping=True
      )
      #Print output for each sequence generated above
    for i, beam in enumerate(generated_text_samples):
        temp = tokenizer.decode(beam, skip_special_tokens=True)
        start_index = temp.find(end_str)
        end_index = start_index + len(end_str)
        result = temp[:start_index]
        s_i = result.find(start_str)
        result = result.replace(start_str,"")
        print("{}: {} |: {}".format(i,result[:s_i],result[s_i:]))
        print()
        
    choice = -1
    choice = input("Enter exit to exit system or the number to the resposne you like  ")
    if choice=="exit":
        flag = False
        continue
    if choice!="exit" and choice in ["0","1","2","3","4"]:
        for i, beam in enumerate(generated_text_samples):
            if i == (int)(choice):
                temp = tokenizer.decode(beam, skip_special_tokens=True)
                start_index = temp.find(end_str)
                end_index = start_index + len(end_str)
                result = temp[:start_index]
                result = result.replace(start_str,"")
                X = result
                
                
    input2 = input("YOU: ")
    X += input2+" "
    print(X)

HI


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


0: HI  |: Hi!

1: HI  |: Ah, hi. Oh my God. Hi!

2: HI  |: Hi!

3: HI  |: Hi.

4: HI  |: Hello, Joey?

Enter exit to exit system or the number to the resposne you like4
YOU:No it is chandler


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


HI Hello, Joey?No it is chandler
0: HI Hello, Joey?No it is chandler  |: Hey Chandler. I'm not gonna miss tonight.

1: HI Hello, Joey?No it is chandler  |: Hey.

2: HI Hello, Joey?No it is chandler  |: What?

3: HI Hello, Joey?No it is chandler  |: Hi!

4: HI Hello, Joey?No it is chandler  |: Hi!

Enter exit to exit system or the number to the resposne you like0
YOU:You better not


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


HI Hello, Joey?No it is chandler Hey Chandler. I'm not gonna miss tonight.You better not
0: HI Hello, Joey?No it is chandler Hey Chandler. I'm not gonna miss tonight.You better not  |: That's right! No-no you don't mind if we get to do that tomorrow night!

1: HI Hello, Joey?No it is chandler Hey Chandler. I'm not gonna miss tonight.You better not  |: All right okay, what's wrong with you guys anyway?!

2: HI Hello, Joey?No it is chandler Hey Chandler. I'm not gonna miss tonight.You better not  |: But you can't stay a week without saying that.

3: HI Hello, Joey?No it is chandler Hey Chandler. I'm not gonna miss tonight.You better not  |: I didn't want to ruin this for you.

4: HI Hello, Joey?No it is chandler Hey Chandler. I'm not gonna miss tonight.You better not  |: I can't believe this!

Enter exit to exit system or the number to the resposne you likeexit
